# Building a QSAR model for Cruzipain (CZP)

## Part 3: Clasification Models

In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn import pipeline
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, matthews_corrcoef
import xgboost as xgb
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler
from sklearn.model_selection import RepeatedStratifiedKFold

#### Datasets

Tenemos 7 dataset para trabajar con distintos tipos de descriptores

In [2]:
rdkit = pd.read_csv('CZP_bioactivity_preprocessed_data_RDKit_2D.csv') # Dataset descriptores en 2D de RDKit
#maccs = pd.read_csv('CZP_maccs.csv') # Dataset con maccs keys
#ecfp4 = pd.read_csv('CZP_ecfp4.csv') # Dataset con Morgan fingerprints radio 2
#ecfp6 = pd.read_csv('CZP_ecfp6.csv') # Dataset con Morgan fingerprints radio 3
#fcfp4 = pd.read_csv('CZP_fcfp4.csv') # Dataset con FeatMorgan fingerprints radio 2
#fcfp6 = pd.read_csv('CZP_fcfp6.csv') # Dataset con FeatMorgan fingerprints radio 3
#padel = pd.read_csv('CZP_padeldesc.csv') # Dataset con Padel Descriptors

#### RDKit 2D Descriptors Preprocesing

In [3]:
rdkit.drop(['molecule_chembl_id', 'canonical_smiles','pIC50'], axis=1, inplace=True)

In [4]:
rdkit.shape

(546, 201)

In [5]:
cor_matrix = rdkit.corr().abs()

In [6]:
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))

In [7]:
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]
print()
print(to_drop)


['MaxAbsEStateIndex', 'HeavyAtomMolWt', 'ExactMolWt', 'NumValenceElectrons', 'FpDensityMorgan2', 'Chi0', 'Chi0n', 'Chi0v', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', 'Chi2v', 'Chi3n', 'Chi3v', 'Chi4n', 'Chi4v', 'Kappa1', 'Kappa2', 'LabuteASA', 'HeavyAtomCount', 'MolMR', 'fr_COO2', 'fr_C_O_noCOO', 'fr_Nhpyrrole', 'fr_benzene', 'fr_nitro_arom', 'fr_phenol', 'fr_phenol_noOrthoHbond']


In [8]:
rdkit2 = rdkit.drop(to_drop, axis=1)

In [9]:
rdkit2.shape

(546, 172)

In [10]:
bio_mapping = {'active': 0, 'inactive': 1}
rdkit2.bioactivity_class = rdkit2['bioactivity_class'].map(bio_mapping)

In [11]:
rdkit2.describe()

,bioactivity_class,MaxEStateIndex,MinEStateIndex,MinAbsEStateIndex,qed,MolWt,NumRadicalElectrons,MaxPartialCharge,MinPartialCharge,MaxAbsPartialCharge,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
count,546.000000,546.000000,546.000000,546.000000,546.000000,546.000000,546.0,546.000000,546.000000,546.000000,...,546.000000,546.000000,546.000000,546.0,546.000000,546.000000,546.0,546.000000,546.000000,546.000000
mean,0.415751,10.851119,-1.084521,0.156397,0.518640,361.670203,0.0,0.277216,-0.409089,0.419628,...,0.082418,0.020147,0.080586,0.0,0.003663,0.058608,0.0,0.054945,0.054945,0.049451
std,0.493303,2.963860,1.657574,0.162654,0.193121,106.419981,0.0,0.091947,0.082515,0.081396,...,0.318515,0.140630,0.272448,0.0,0.060467,0.235105,0.0,0.228082,0.343622,0.217006
min,0.000000,2.627060,-5.001750,0.000000,0.128283,147.133000,0.0,0.056730,-1.000000,0.153678,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
25%,0.000000,9.281164,-1.056486,0.053287,0.366079,285.480250,0.0,0.203425,-0.479438,0.368346,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
50%,0.000000,12.047829,-0.465912,0.108120,0.510446,337.810000,0.0,0.263259,-0.412099,0.412109,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
75%,1.000000,12.736188,0.063163,0.188652,0.673087,418.172250,0.0,0.330359,-0.351315,0.481069,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
max,1.000000,14.818185,0.769311,0.769311,0.939592,705.647000,0.0,1.000000,-0.153678,1.000000,...,2.000000,1.000000,1.000000,0.0,1.000000,1.000000,0.0,1.000000,4.000000,1.000000


In [12]:
X_rdkit = rdkit2.drop('bioactivity_class', axis=1)
y = rdkit2.bioactivity_class

In [14]:
# randomly select 30% of compounds as test set
X_train_rdkit, X_test_rdkit, y_train_rdkit, y_test_rdkit = train_test_split(X_rdkit, y, test_size=0.20, random_state=42, stratify=y)

In [15]:
df_columns = X_rdkit.columns.tolist()

In [16]:
# obtain scale object which can be further applied to scale any data to fit the training set
scaler = StandardScaler()

X_train_scaled_rdkit = pd.DataFrame(scaler.fit_transform(X_train_rdkit), columns=df_columns)


In [17]:
X_train_scaled_rdkit.describe()

,MaxEStateIndex,MinEStateIndex,MinAbsEStateIndex,qed,MolWt,NumRadicalElectrons,MaxPartialCharge,MinPartialCharge,MaxAbsPartialCharge,MinAbsPartialCharge,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
count,4.360000e+02,436.000000,4.360000e+02,4.360000e+02,4.360000e+02,436.0,4.360000e+02,4.360000e+02,4.360000e+02,4.360000e+02,...,436.000000,4.360000e+02,4.360000e+02,436.0,4.360000e+02,4.360000e+02,436.0,4.360000e+02,4.360000e+02,4.360000e+02
mean,3.300112e-16,0.000000,-9.778111e-17,8.453992e-17,-2.281559e-16,0.0,6.162247e-16,-5.907609e-17,-1.344490e-16,2.363044e-16,...,0.000000,-1.222264e-17,-2.444528e-17,0.0,1.629685e-17,-1.018553e-17,0.0,6.518741e-17,-4.074213e-17,8.046571e-17
std,1.001149e+00,1.001149,1.001149e+00,1.001149e+00,1.001149e+00,0.0,1.001149e+00,1.001149e+00,1.001149e+00,1.001149e+00,...,1.001149,1.001149e+00,1.001149e+00,0.0,1.001149e+00,1.001149e+00,0.0,1.001149e+00,1.001149e+00,1.001149e+00
min,-2.785627e+00,-2.319140,-9.740751e-01,-1.994039e+00,-2.006219e+00,0.0,-2.355055e+00,-7.509764e+00,-3.394007e+00,-2.795449e+00,...,-0.247314,-1.451802e-01,-3.089942e-01,0.0,-6.788442e-02,-2.466318e-01,0.0,-2.359874e-01,-1.551300e-01,-2.249498e-01
25%,-4.546373e-01,-0.012638,-6.314268e-01,-7.809384e-01,-7.206713e-01,0.0,-8.011626e-01,-9.068580e-01,-5.717384e-01,-8.478967e-01,...,-0.247314,-1.451802e-01,-3.089942e-01,0.0,-6.788442e-02,-2.466318e-01,0.0,-2.359874e-01,-1.551300e-01,-2.249498e-01
50%,3.985139e-01,0.392362,-2.834070e-01,-3.073269e-02,-2.222150e-01,0.0,-1.056893e-01,-5.271615e-02,-9.698253e-02,-5.755924e-02,...,-0.247314,-1.451802e-01,-3.089942e-01,0.0,-6.788442e-02,-2.466318e-01,0.0,-2.359874e-01,-1.551300e-01,-2.249498e-01
75%,6.313195e-01,0.691997,1.865427e-01,8.187827e-01,5.340386e-01,0.0,5.434114e-01,7.182780e-01,7.619930e-01,7.365461e-01,...,-0.247314,-1.451802e-01,-3.089942e-01,0.0,-6.788442e-02,-2.466318e-01,0.0,-2.359874e-01,-1.551300e-01,-2.249498e-01
max,1.333937e+00,1.133616,3.940840e+00,2.066645e+00,3.198785e+00,0.0,7.636705e+00,3.225150e+00,7.403238e+00,3.222939e+00,...,6.095553,6.887993e+00,3.236307e+00,0.0,1.473092e+01,4.054627e+00,0.0,4.237514e+00,1.066674e+01,4.445436e+00


In [18]:
# scale descriptors of the test set compounds
X_test_scaled_rdkit = pd.DataFrame(scaler.transform(X_test_rdkit), columns=df_columns)

## Modeling

#### **Random Forest**

In [31]:
RF=RandomForestClassifier()

In [32]:
parameters = {
    'n_estimators': [125,150,175],
    'max_features': ['auto', 'sqrt', 'log2', None],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
              }

In [33]:
rf_gsearch = GridSearchCV(RF, parameters, n_jobs= 2, verbose=2,cv=5)

In [35]:
rf_gsearch.fit(X_train_scaled_rdkit, y_train_rdkit) 

Fitting 5 folds for each of 120 candidates, totalling 600 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:   10.6s
[Parallel(n_jobs=2)]: Done 158 tasks      | elapsed:   50.9s
[Parallel(n_jobs=2)]: Done 361 tasks      | elapsed:  2.1min
[Parallel(n_jobs=2)]: Done 600 out of 600 | elapsed:  3.7min finished


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=2,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [4, 5, 6, 7, 8],
                         'max_features': ['auto', 'sqrt', 'log2', None],
                         'n_estimators': [125, 150, 175]},
             verbose=2)

In [24]:
#Predictions and report
y_test_pred_rf = rf_gsearch.predict(X_scaled_test_rdkit)
y_train_pred_rf = rf_gsearch.predict(X_scaled_train_rdkit)
print(classification_report(y_test_pred_rf,y_test_rdkit))

              precision    recall  f1-score   support

           0       0.83      0.83      0.83        64
           1       0.76      0.76      0.76        46

    accuracy                           0.80       110
   macro avg       0.79      0.79      0.79       110
weighted avg       0.80      0.80      0.80       110



In [25]:
confusion_matrix(y_test_rdkit, y_test_pred_rf)

array([[53, 11],
       [11, 35]])

In [26]:
print('Best score and parameter combination = ')

print(rf_gsearch.best_score_)    
print(rf_gsearch.best_params_)   

Best score and parameter combination = 
0.8278735632183907
{'criterion': 'entropy', 'max_depth': 7, 'max_features': 'log2', 'n_estimators': 150}


In [27]:
round(accuracy_score(y_test_rdkit, y_test_pred_rf),2)

0.8

In [28]:
matthews_corrcoef(y_test_rdkit, y_test_pred_rf)

0.5889945652173914

Using optuna

### XGBoost 

In [29]:
def objective(trial: Trial,X,y) -> float:
    
    #joblib.dump(study, 'study.pkl')
    
    train_X,test_X,train_y,test_y = train_test_split(X, Y, test_size = 0.30,random_state = 101)

    param = {
                "n_estimators" : trial.suggest_int('n_estimators', 0, 1000),
                'max_depth':trial.suggest_int('max_depth', 2, 25),
                'reg_alpha':trial.suggest_int('reg_alpha', 0, 5),
                'reg_lambda':trial.suggest_int('reg_lambda', 0, 5),
                'min_child_weight':trial.suggest_int('min_child_weight', 0, 5),
                'gamma':trial.suggest_int('gamma', 0, 5),
                'learning_rate':trial.suggest_loguniform('learning_rate',0.005,0.5),
                'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree',0.1,1,0.01),
                'nthread' : -1
            }
    model = XGBClassifier(**param)

    model.fit(train_X,train_y)

    return cross_val_score(model,test_X,test_y).mean()

In [3]:
xgb_clf = xgb.XGBClassifier(n_estimators=250, objective='binary:logistic', silent=True, nthread=1)

In [ ]:
params_xgb = { 
    "learning_rate": [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
    "max_depth": [ 3, 4, 5, 6, 8, 10, 12, 15],
    "min_child_weight" : [ 1, 3, 5, 7 ],
    "gamma" : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
    "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
        
}

In [4]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

In [64]:
scores = cross_val_score(xgb_clf, X_train_rdkit, y_train_rdkit, scoring='accuracy', cv=cv, n_jobs=-1)

In [65]:
print('Mean Accuracy: %.5f' % np.mean(scores))

Mean Accuracy: 0.79856


## ExtraTree Classifier

In [31]:
etc = ExtraTreesClassifier(n_estimators=100, random_state=42)

In [32]:
etc.fit(X_train_rdkit, y_train_rdkit)

ExtraTreesClassifier(random_state=42)

In [34]:
y_test_pred_etc = etc.predict(X_test_rdkit)
y_train_pred_etc = etc.predict(X_train_rdkit)
print(classification_report(y_test_pred_etc, y_test_rdkit))

              precision    recall  f1-score   support

      active       0.90      0.88      0.89        98
    inactive       0.82      0.85      0.84        66

    accuracy                           0.87       164
   macro avg       0.86      0.86      0.86       164
weighted avg       0.87      0.87      0.87       164



In [45]:
params_etc = {
    'n_estimators': [100,125,150,175],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [6,7,8, 9,10,None],
    'criterion' :['gini', 'entropy']
    }

In [46]:
etc_gsearch = GridSearchCV(etc, params_etc, n_jobs= 2, cv=10, verbose=2, return_train_score=False)

In [47]:
etc_gsearch.fit(X_train_rdkit, y_train_rdkit)

Fitting 10 folds for each of 144 candidates, totalling 1440 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    7.8s
[Parallel(n_jobs=2)]: Done 158 tasks      | elapsed:   23.7s
[Parallel(n_jobs=2)]: Done 361 tasks      | elapsed:   53.6s
[Parallel(n_jobs=2)]: Done 644 tasks      | elapsed:  1.6min
[Parallel(n_jobs=2)]: Done 1009 tasks      | elapsed:  2.4min
[Parallel(n_jobs=2)]: Done 1440 out of 1440 | elapsed:  3.5min finished


GridSearchCV(cv=10, estimator=ExtraTreesClassifier(random_state=42), n_jobs=2,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [6, 7, 8, 9, 10, None],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [100, 125, 150, 175]},
             verbose=2)

In [48]:
etc_gsearch.best_params_

{'criterion': 'gini',
 'max_depth': None,
 'max_features': 'log2',
 'n_estimators': 100}

In [49]:
y_test_pred_etc_gsearch = etc_gsearch.predict(X_test_rdkit)
y_train_pred_etc_gsearch = etc_gsearch.predict(X_train_rdkit)
print(classification_report(y_test_pred_etc_gsearch, y_test_rdkit))

              precision    recall  f1-score   support

      active       0.90      0.88      0.89        98
    inactive       0.82      0.85      0.84        66

    accuracy                           0.87       164
   macro avg       0.86      0.86      0.86       164
weighted avg       0.87      0.87      0.87       164



In [50]:
confusion_matrix(y_test_rdkit, y_test_pred_etc_gsearch)

array([[86, 10],
       [12, 56]])

In [51]:
round(accuracy_score(y_test_rdkit, y_test_pred_etc_gsearch),2)

0.87

In [54]:
round(accuracy_score(y_train_rdkit, y_train_pred_etc_gsearch),2)

1.0

In [53]:
round(matthews_corrcoef(y_test_rdkit, y_test_pred_etc_gsearch),2)

0.72

In [55]:
round(matthews_corrcoef(y_train_rdkit, y_train_pred_etc_gsearch),2)

1.0

In [ ]:
# Definimos listas vacias donde vamos a "appendear" (agregar) los valores
accuracy_test = []
accuracy_train = []

# Calculamos el accuracy sobre el test set
for prediccion_test in ada_clf.staged_predict(X_test):
    accuracy_test.append(metrics.accuracy_score(prediccion_test,y_test))
    
# Calculamos el accuracy sobre el training set    
for prediccion_train in ada_clf.staged_predict(X_train):  
    accuracy_train.append(metrics.accuracy_score(prediccion_train,y_train))
    
plt.plot(range(1, len(accuracy_test) + 1), accuracy_test, label = 'Test')
plt.plot(range(1, len(accuracy_test) + 1), accuracy_train, label = 'Train')
plt.legend()
plt.ylabel('Accuracy en el test set')
plt.xlabel('Número de árboles')